In [1]:
file_names = !ls ../data/raw_splits/train/

In [2]:
chunk_names = []
for i in file_names:
    if "chunk_" in i:
        chunk_names.append(i)
chunk_names = sorted(chunk_names)

In [3]:
import pandas as pd
import numpy as np
import datetime
import imp
import scripts as scr

In [4]:
imp.reload(scr)

<module 'scripts' from '/data/hse_nis/notebooks/scripts.py'>

In [5]:
# data = pd.read_feather('../data/raw_splits/train/{}'.format(chunk_names[0]))

In [6]:
# data.columns

In [7]:
include_channels = ['Перевод частному лицу', 'Оплата услуг', 'Перевод между своими счетами и картами']

In [8]:
include_channels

['Перевод частному лицу',
 'Оплата услуг',
 'Перевод между своими счетами и картами']

In [9]:
start_date = 20171029
end_date = 20171128

In [10]:
#del feat_test
# feat_test = scr.features_handler(
#     chunk_names=[chunk_names[9]],
#     calc_feat = scr.calc_base_features,
#     query=("event_description in {incl}  and short_date > {start} and short_date < {end}"
#            .format(incl=include_channels, start=start_date, end=end_date)))

/data/hse_nis/notebooks/scripts.py:96: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  feat_matrix['recip_card_age'] = [x.days if type(x) is not pd.tslib.NaTType else 912321 for x in (data.event_time - data.cdf_s_124)]


In [48]:
import multiprocessing as mlp

In [49]:
def get_data(chunk_name):
    #del feat_test
    feat_test = scr.features_handler(
        chunk_names=[chunk_name],
        calc_feat = scr.calc_base_features,
        query=("event_description in {incl}  and short_date > {start} and short_date < {end}"
               .format(incl=include_channels, start=start_date, end=end_date)))
    return feat_test

In [53]:
# жрет вплоть более 70 гигов оперативки, осторожнее
with mlp.Pool(processes=15) as pool:
    results = pool.map(get_data, chunk_names)

In [54]:
total_df = pd.DataFrame()
for i, df in enumerate(results):
    total_df = pd.concat([total_df, df])
    results[i] = 'bye'

In [58]:
total_df['short_date'] = total_df.event_time.apply(lambda x: x.date())
total_df.rename(columns={"custom_mark": "label"}, inplace=True)
total_df['label'] = total_df.label.apply(lambda x: scr.cust_mark_to_class(x))

In [66]:
total_df = total_df.query("label != -1")

In [74]:
global_mean = np.mean(total_df['label'])

In [79]:
n_uniques = np.apply_along_axis(lambda x: len(np.unique(x)), axis=0, arr=total_df)

In [100]:
# total_df.to_csv("../data/coms_sep/train.csv")
total_df = pd.read_csv("../data/coms_sep/train.csv", index_col=0)

In [101]:
def map_counter(data, field_name, target_name, alpha, global_mean=None):
    if global_mean is None:
        global_mean = data[target_name].mean()
    counters = data.groupby(field_name)[target_name].mean()
    n_counters = data.groupby(field_name)[target_name].count()
    out = ((data[field_name].map(counters) + global_mean * alpha)/ \
           (data[field_name].map(n_counters) + alpha))
    return out

In [110]:
n_uniques

array([77143, 44679,     2, 75294, 13327, 18268,     3,     3,     1,
           3,     2, 34704, 25670,   605,     2,   704,     8,     8,
           8,     8,     8,   689,     8,     1,  5304,     2,   704,
       13327,    14, 19904,    30])

In [103]:
total_df.columns

Index(['event_id', 'user_id', 'label', 'event_time', 'amount', 'client_age',
       'cat_new_ip', 'cat_new_prov', 'channel_op', 'op_type', 'recip_age',
       'age_diff', 'cumulative_sum_total', 'data_i_120', 'relative',
       'know_recip_power', 'cdf_s_127', 'cdf_s_135', 'cdf_s_130', 'cdf_s_129',
       'cdf_s_134', 'data_i_154', 'cdf_s_133', 'know_recip_card_age',
       'recip_card_age', 'one_region', 'krp_pow2', 'log_amount', 'ip_isp',
       'amnt2chnls', 'short_date'],
      dtype='object')

In [105]:
for col in total_df.columns:
    col_name = total_df.columns[i]
    if col_name not in ['event_id', "user_id", "event_time", "label", "short_date"]:
        total_df[col] = map_counter(total_df, col, 'label',
                                    alpha=1000,
                                    global_mean=global_mean)

In [108]:
#total_df.to_csv("../data/coms_sep/train_cntrs.csv")
#total_df = pd.read_csv("../data/coms_sep/train_cntrs.csv", index_col=0)

In [1]:
total_df.label.value_count

NameError: name 'total_df' is not defined